### 해당 파일을 Base로 두고, duplicate하여 사용 권장드립니다
#

# 0. Setup

In [1]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='c0b63e8d-5403-4e7c-9093-e276d9269f8d', project_access_token='p-2+wAu+98G+n6mClSRjwoNKiw==;IlK3hGo3VPIK1GrqxfM0SQ==:n8wgYw8viYRnnSbyPCq9dMOV++gxofZ87T6ZaRqSSVa7t7HHdhZmBjQh6OTWjtZS8gSJfvutaX+4+PacEwwGP1bjp1IlVhjFfw==')
pc = project.project_context

from ibm_watson_studio_lib import access_project_or_space
wslib = access_project_or_space({'token':'p-2+wAu+98G+n6mClSRjwoNKiw==;IlK3hGo3VPIK1GrqxfM0SQ==:n8wgYw8viYRnnSbyPCq9dMOV++gxofZ87T6ZaRqSSVa7t7HHdhZmBjQh6OTWjtZS8gSJfvutaX+4+PacEwwGP1bjp1IlVhjFfw=='})

In [2]:
# !pip install python-dotenv

In [3]:
from dotenv import load_dotenv

env_file = 'env'
wslib.download_file(env_file)
load_dotenv(env_file)

True

# 1. install libs & load pdf from COS

In [4]:
## Notebook Runtime을 변경했기 때문에 수동으로 설치할 필요 없음
## 설치 이후에 Kernel Restart 필요
#
!pip -q install langchain pymilvus sentence_transformers pypdf

In [5]:
import types
import pandas as pd
import ibm_boto3
from botocore.client import Config
import os

def __iter__(self): return 0


STORAGE_API_KEY_ID = os.getenv("IBM_STR_API", None)
STORAGE_SERVICE_INSTANCE_ID = os.getenv("IBM_STR_INSTANCE", None)
STORAGE_AUTH_ENDPOINT = os.getenv("IBM_STR_AUTH", None)
STORAGE_ENDPOINT_URL = os.getenv("IBM_STR_URL", None)


paper_cos_client = ibm_boto3.client(
    service_name='s3',
    ibm_api_key_id=STORAGE_API_KEY_ID,
    ibm_service_instance_id=STORAGE_SERVICE_INSTANCE_ID,
    ibm_auth_endpoint=STORAGE_AUTH_ENDPOINT,
    config=Config(signature_version='oauth'),
    endpoint_url=STORAGE_ENDPOINT_URL
)

    
# Using IBM Cloud Object Storage to save & get documents    
# Specify the bucket name
bucket_name = 'hae-ko'

# List objects within the bucket
objects = paper_cos_client.list_objects_v2(Bucket=bucket_name)

# Make path to save PDF files
!mkdir -p PDF


In [6]:
cd PDF/

/home/wsuser/work/PDF


현재, IBM Cloud Object Storage에 파일 모두 업로드 한 상태임. \
원하는 문서에 따라 전체 또는 한 개 파일을 지정해서 불러와 test해볼 수 있음

In [9]:
# Your data file is loaded into a botocore.response.StreamingBody object.
# Refer to the documentation of ibm_boto3 and pandas for other possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/

# Choose option between go_with_selected | go_with_all based on the trial contents

def go_with_selected(file_name):
    streaming_body  = paper_cos_client.get_object(Bucket=bucket_name, Key=file_name)['Body']
    
    if not hasattr(streaming_body , "__iter__"): streaming_body .__iter__ = types.MethodType( __iter__, streaming_body  ) 
    
    with open('./PDF/'+file_name, 'wb') as f:
        f.write(streaming_body.read())
        
def go_with_all():    
    for obj in objects.get('Contents', []):
        key = obj['Key']

        # Check if the object is a PDF file
        if key.endswith('.pdf'):
            # Retrieve the object
            streaming_body = paper_cos_client.get_object(Bucket=bucket_name, Key=key)['Body']

            # Add missing __iter__ method if not present
            if not hasattr(streaming_body, "__iter__"): 
                streaming_body.__iter__ = types.MethodType(__iter__, streaming_body)

            with open('./PDF/'+key, 'wb') as f:
                f.write(streaming_body.read())

In [10]:
# Choose between two
#
!mkdir -p PDF

go_with_selected('2023_G70_Ko.pdf')
# go_with_all()

In [11]:
ls -lh ./PDF/

total 8.2M
-rw-rw---- 1 wsuser wscommon 8.2M Dec 15 03:32 2023_G70_Ko.pdf


# 2. Document Ingestion

a. connect to Milvus

In [99]:
import os, time
from dotenv import load_dotenv

from pymilvus import connections
from pymilvus import Collection, CollectionSchema
from pymilvus import FieldSchema, DataType
from pymilvus import utility


from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Milvus
from langchain.text_splitter import RecursiveCharacterTextSplitter


pem_file = 'cert.pem'
wslib.download_file(pem_file)

COLLECTION_NAME = "test01"
# MILVUS_PW = os.getenv("MILVUS_PASSWORD", None)

connections.connect("default", 
                    host="161.156.203.216", 
                    port="8080", 
                    secure=True, 
                    server_pem_path=pem_file, 
                    server_name="localhost",
                    user="root",
                    password="4XYg2XK6sMU4UuBEjHq4EhYE8mSFO3Qq")

b. check if collection exist

In [100]:
## check if collection exist
#
has = utility.has_collection(COLLECTION_NAME)
    
print(f"Does collection hello_milvus exist in Milvus: {has}")

Does collection hello_milvus exist in Milvus: True


In [102]:
## create collection
#
fields = [
    FieldSchema(name = "id", dtype = DataType.INT64, description = "Ids", is_primary = True, auto_id = False),
    FieldSchema(name = "content", dtype = DataType.VARCHAR, description = "Content texts", max_length = 768*8),
    FieldSchema(name = "embedding", dtype = DataType.FLOAT_VECTOR, description = "Embedding vectors", dim = 384)
]
schema = CollectionSchema(fields = fields, description = "content collection")
collection = Collection(name = COLLECTION_NAME, schema = schema)

## create index 
#
index_params = {
    "index_type": "IVF_FLAT",
    "metric_type": "L2",
    "params": {"nlist": 1024}
}
collection.create_index(field_name = "embedding", index_params = index_params)

Status(code=0, message=)

c. ingest doccument (a pdf file)


In [122]:
## load embedding model
#
model_name = 'sentence-transformers/all-MiniLM-L6-v2' # Try out with different model
embeddings = HuggingFaceEmbeddings(model_name=model_name)

## text splitter parmas
#
chunk_size = 1000
chunk_overlap = 150
separator = "\n"

## create text splicter instance
#
r_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

In [123]:
## load PDF
#
def load_PDF(filename):
    loader = PyPDFLoader(filename)
    pages = loader.load()
    
    len(pages)
    
    docs = r_splitter.split_documents(pages)
    
    return docs

## ingest PDF to Milvus
#
def ingest_PDF_Milvus(docs):
    
    start = time.time()
    data = [[], [], []]
    if docs:
        for idx, text in enumerate(docs):
            data[0].append(idx)
            data[1].append(text.page_content)
            data[2].append(embeddings.embed_query(text.page_content))
            

    collection.load()
    collection.insert(data)
    collection.flush()

    print(f"\nEnlapse time: {time.time() - start} sec")

In [124]:
## get list of files (Path = ./PDF/*.pdf)) 
#
PATH = "./PDF"
# PATH="./"
PDF_list = os.listdir(PATH)

for idx, pdf_file in enumerate(PDF_list):
    
    print(f"{idx+1} - {pdf_file}\n")
    
    pdf_path = os.path.join(PATH, pdf_file)
    docs = load_PDF(pdf_path)
    
    ingest_PDF_Milvus(docs)

1 - 2023_G70_Ko.pdf


Enlapse time: 84.63170456886292 sec


In [125]:
utility.list_collections()

['HAE_embedding_01', 'test01']

# 4. Run Similarity Search

In [130]:
USER_INPUT = '에어백 주의사항'

In [131]:
embedded_query = embeddings.embed_query(USER_INPUT)
print(embedded_query[:10])

[-0.025476165115833282, 0.084379181265831, 0.014332981780171394, 0.02265176922082901, -0.02761511690914631, -0.10611443221569061, 0.022642523050308228, 0.07290681451559067, -0.02983572892844677, -0.012340664863586426]


In [132]:
## search params
#
search_params = {
    "metric_type": "L2", 
    "offset": 5, 
    "ignore_growing": False, 
    "params": {"nprobe": 10}
}

## conduct search
#
results = collection.search(
    data=[embedded_query], 
    anns_field="embedding", 
    param=search_params,
    limit=3,
    expr=None,
    output_fields=['content'],
    consistency_level="Strong"
)

In [133]:
results

['["id: 309, distance: 0.8592061996459961, entity: {\'content\': \'지고 하이빔 보조가 해제됩니다. \\\\n•하이빔 보조가 작동하여 상향등이 켜진 상\\\\n태에서 다음과 같은 상황이 발생하면 하향\\\\n등 상태로 자동 전환될 수 있습니다. 다양\\\\n한 운전 환경을 고려하여 상황에 맞게 기\\\\n능을 사용하십시오.\\\\n- 다가오는 차량의 전조등을 감지할 경우\\\\n- 앞서가는 차량의 후미등을 감지할 경우\\\\n- 자전거 및 이륜차의 전조등 또는 후미등\\\\n을 감지할 경우\\\\n- 상향등을 켜지 않아도 될 만큼 주위가\\\\n밝을 경우\\\\n- 전방에 가로등이나 기타 광원이 있을 경\\\\n우\\\\nIK_G70_KO.book  Page 75\'}", "id: 701, distance: 0.8600937724113464, entity: {\'content\': \'즉시 제거하십시오.\\\\n҃Ҋ\\\\n \\\\n실내등은 청소하기 전에, 실내등이 꺼졌는지\\\\n확인하여 화상을 입거나 전기 충격을 받지\\\\n않도록 주의하십시오.\\\\n \\\\n \\\\u0ab1\\\\u0a44\\\\n \\\\n•가죽제품(스티어링 휠, 시트 등) 청소 시에\\\\n는 중성 세제 또는 알코올이 소량 함유된\\\\n세척제를 사용하십시오. 강한 산성/알칼\\\\n리성 및 알코올이 다량 함유된 세척제를\\\\n사용할 경우, 변색 및 표면 벗겨짐 등의 손상이 발생할 수 있습니다.\\\\n•과도한 오염 제거는 내장재 또는 카매트(\\\\n카펫)에 손상을 줄 수 있으니 주의하십시\\\\n오.\\\\n \\\\n안전벨트의 청소\\\\n내장재 또는 카펫용으로 추천된 부드러운 비\\\\n누액을 사용하여 안전벨트를 청소하십시오.\\\\n표백처리를 하거나 재염색을 하지 마십시오.\\\\n내부 창문의 청소\\\\n차량의 창문 내측 표면에 서리가 끼면(즉 기\\\\n름끼나 왁스끼 묻은 필름으로 덮이면) 창문\\\\n세척기로

In [120]:
## get the IDs of all returned hits 
#
print(f"1) doc ID: {results[0].ids}\n")

## get the distances to the query vector from all returned hits
#
print(f"2) distance: {results[0].distances}\n")

1) doc ID: []

2) distance: []



In [121]:
## create documents varialbe for RAG pattern
#

documents = []
for raw_result in results:
    for result in raw_result:
        documents.append(result.entity.get("content"))

# 05. Call LLM model from watsonx.ai

a. Get Credential for watsonx

In [59]:
from langchain.embeddings import HuggingFaceEmbeddings
from ibm_watson_machine_learning import APIClient

env_file = 'env'
wslib.download_file(env_file)

load_dotenv(env_file)

WATSONX_URL = os.getenv("IBM_CLOUD_URL", None)
WATSONX_API_KEY = os.getenv("API_KEY", None)
WATSONX_PROJECT_ID = os.getenv("PROJECT_ID", None)

watsonx_credentials = {
    "url": WATSONX_URL,
    "apikey": WATSONX_API_KEY
}

client = APIClient(watsonx_credentials)

In [75]:
initial_prompt="""
에이전트: 인스트럭션을 기다리고 있습니다.\n
사용자: 여기 당신의 지시사항입니다:\n
1. 사용자의 질문에 대답하기 위해 사용할 문서가 주어질 것입니다.\n
2. 문서에 있는 정보를 사용하여 다음 대답을 생성해야 합니다.\n
3. 답을 찾을 수 없다면 모르겠다고 답하세요.\n
4. 답변은 길지 않아야 하며 2문장 정도여야 합니다.\n
5. 답변을 반복해서는 안됩니다.\n
6. 다른 지식을 사용하지 마세요.\n
7. 답변은 읽기 좋게 짧은 문장으로만 작성하세요.\n
지시사항을 다시 말해보세요. 이해했는지 알고 싶습니다.\n\n
에이전트: 알겠습니다. 제 지시사항은 다음과 같습니다:\n
1. 사용자의 질문에 대답하기 위해 사용할 문서가 주어질 것입니다.\n
2. 문서에 있는 정보를 사용하여 다음 대답을 생성해야 합니다.\n
3. 답을 찾을 수 없다면 모르겠습니다로 대답하겠습니다.\n
4. 제 대답은 길지 않아야 하며 2 문장 정도여야 합니다.\n
5. 답변을 반복해서는 안됩니다.\n
6. 주어진 정보 외에는 다른 지식을 사용하지 않을 것입니다.\n
7. 답변은 읽기 편하게 간결하고 깔끔한 문장 형태로만 출력하겠습니다.\n
모두 준비되었습니다. 이제 문서를 보여주세요.\n\n
사용자: 여기 문서가 있습니다:\n"""

In [86]:
def prompt_template(initial_prompt, context, USER_INPUT):
    return (
        initial_prompt
        + f"{context}\n\n"
        + f"에이전트: 문서로부터 질문에 대한 답을 할 준비가 되었습니다.\n\n"
        + f"사용자: {USER_INPUT}\n\n"
        + "에이전트: "
    )

In [87]:
context = "\n\n\n".join(documents)

prompt = prompt_template(initial_prompt, context, USER_INPUT)
print(prompt)


에이전트: 인스트럭션을 기다리고 있습니다.

사용자: 여기 당신의 지시사항입니다:

1. 사용자의 질문에 대답하기 위해 사용할 문서가 주어질 것입니다.

2. 문서에 있는 정보를 사용하여 다음 대답을 생성해야 합니다.

3. 답을 찾을 수 없다면 모르겠다고 답하세요.

4. 답변은 길지 않아야 하며 2문장 정도여야 합니다.

5. 답변을 반복해서는 안됩니다.

6. 다른 지식을 사용하지 마세요.

7. 답변은 읽기 좋게 짧은 문장으로만 작성하세요.

지시사항을 다시 말해보세요. 이해했는지 알고 싶습니다.


에이전트: 알겠습니다. 제 지시사항은 다음과 같습니다:

1. 사용자의 질문에 대답하기 위해 사용할 문서가 주어질 것입니다.

2. 문서에 있는 정보를 사용하여 다음 대답을 생성해야 합니다.

3. 답을 찾을 수 없다면 모르겠습니다로 대답하겠습니다.

4. 제 대답은 길지 않아야 하며 2 문장 정도여야 합니다.

5. 답변을 반복해서는 안됩니다.

6. 주어진 정보 외에는 다른 지식을 사용하지 않을 것입니다.

7. 답변은 읽기 편하게 간결하고 깔끔한 문장 형태로만 출력하겠습니다.

모두 준비되었습니다. 이제 문서를 보여주세요.


사용자: 여기 문서가 있습니다:
십시오.
•이 차량에는 운전자의 운전 이력을 기반으
로 엔진 오일의 수명을 예측하고, 엔진 오
일 교체 알림 정보를 제공하는 오일 수명
관리 시스템이 적용되어 있습니다.
- 운전자의 운전 가혹도에 따라 엔진 오일
의 열화도가 증가될 경우에는 엔진 오일
의 정상 교체 주기 이전에 엔진 오일 잔여 수명 경고 알람이 계기판에 나타나
며, 가까운 정비 사업소를 방문하여 엔
진 오일을 교체하십시오.
- 오일 수명 관리 시스템은 당사 추천 엔
진 오일을 사용했을 때 정상으로 작동하
며, 기타 오일을 사용 시에는 가혹조건
에 해당하여 가혹조건 오일 교환 점검표
에 맞추어 교환이 필요합니다. 또한 본시스템은 엔진 오일이 정상적으로 주입
되어 있음을 전제하므로, 정기적으로 엔진 오일의 양을 점검하시기 바랍니다.

b. Generate Response

In [91]:
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams

model_name='meta-llama/llama-2-70b-chat'

model_params = {
    GenParams.DECODING_METHOD: 'greedy',
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 1024,
    GenParams.TEMPERATURE: 0,
    GenParams.REPETITION_PENALTY: 1.1,
}

model = Model(
    model_id=model_name,
    params=model_params,
    credentials=watsonx_credentials,
    project_id=WATSONX_PROJECT_ID)

In [92]:
generated_text = model.generate_text(prompt=prompt)

print(generated_text)

알겠습니다. 이 차량은 운전자의 운전 이력을 기반으로 엔진 오일의 수


In [93]:
from pprint import pprint 

generated_text = model.generate(prompt=prompt)

pprint(generated_text, indent=4)

{   'created_at': '2023-12-15T04:08:15.251Z',
    'model_id': 'meta-llama/llama-2-70b-chat',
    'results': [   {   'generated_text': '알겠습니다. 이 차량은 운전자의 운전 이력을 기반으로 엔진 오일의 '
                                         '수',
                       'generated_token_count': 58,
                       'input_token_count': 4038,
                       'stop_reason': 'token_limit'}],
    'system': {   'warnings': [   {   'id': 'disclaimer_warning',
                                      'message': 'This model is a Non-IBM '
                                                 'Product governed by a '
                                                 'third-party license that may '
                                                 'impose use restrictions and '
                                                 'other obligations. By using '
                                                 'this model you agree to its '
                                                 'terms as identified in the '
       